---
# Chapter 11
## Interfaces: From Protocols to ABCs

---

## Interfaces and Protocols in Python Culture

---
### Example 11-1: `x` and `y` are public data attributes (see file `vector2d_v0.py` in `Example 9-2` of Chapter 9)

---
### Example 11-2: `x` and `y` reimplemented as properties (see file `vector2d_v3.py` in `Example 9-9` of Chapter 9)

## Python Digs Sequences

---
### Example 11-3: Partial sequence protocol implementation with \_\_getitem\_\_: enough for item access, iteration, and the `in` operator

In [1]:
class Foo:
    def __getitem__(self, index):
        return range(0, 40, 10)[index]

In [2]:
f = Foo()
f[1]

10

In [3]:
for i in f: print(i) 

0
10
20
30


In [4]:
20 in f

True

In [5]:
15 in f

False

---
### Example 11-4: A deck as a sequence of cards (same as Example 1-1)

In [8]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

## Monkey-Patching to implement a Protocol at Runtime

---
### Example 11-5: random.shuffle cannot handle FrenchDeck

In [9]:
from random import shuffle

deck = FrenchDeck()
print(deck[:5])
shuffle(deck)

[Card(rank='2', suit='spades'), Card(rank='3', suit='spades'), Card(rank='4', suit='spades'), Card(rank='5', suit='spades'), Card(rank='6', suit='spades')]


TypeError: 'FrenchDeck' object does not support item assignment

---
### Example 11-6: Monkey patching FrenchDeck to make it mutable and compatible with random.shuffle {continuing from Example 11-5}

In [10]:
def set_card(deck: FrenchDeck, position: int, card: Card) -> None:
    deck._cards[position] = Card

FrenchDeck.__setitem__ = set_card
print(deck[:5])
shuffle(deck)
print(deck[:5])

[Card(rank='2', suit='spades'), Card(rank='3', suit='spades'), Card(rank='4', suit='spades'), Card(rank='5', suit='spades'), Card(rank='6', suit='spades')]
[<class '__main__.Card'>, <class '__main__.Card'>, <class '__main__.Card'>, <class '__main__.Card'>, <class '__main__.Card'>]


---
### Example 11-7: Duck typing to handle a string or an iterable of strings

In [11]:
def ducky_string_handler(field_names):
    try:
        return field_names.replace(',', ' ').split()
    except AttributeError:
        return tuple(field_names)

field_names = 'Hello from python, world'
ducked = ducky_string_handler(field_names)
print(ducked)

field_names = ['Hello', 'from', 'python', 'world']
ducked = ducky_string_handler(field_names)
print(ducked)

field_names = ('Hello', 'from', 'python', 'world',)
ducked = ducky_string_handler(field_names)
print(ducked)


['Hello', 'from', 'python', 'world']
('Hello', 'from', 'python', 'world')
('Hello', 'from', 'python', 'world')


---
### Example 11-8: `frenchdeck2.py`: FrenchDeck2, a subclass of collections.MutableSequence (See file `frenchdeck2.py`)

In [12]:
from frenchdeck2 import FrenchDeck2

In [14]:
import random
deck = FrenchDeck2() 

print(random.choice(deck))
random.shuffle(deck)
print(deck[:5])

Card(rank='K', suit='diamonds')
[Card(rank='4', suit='spades'), Card(rank='Q', suit='hears'), Card(rank='4', suit='clubs'), Card(rank='2', suit='hears'), Card(rank='6', suit='spades')]


## Defining and Using an ABC

---
### Example 11-9: tombola.py: Tombola is an ABC (see file `tombola.py`)

In [15]:

from tombola import Tombola

---
### Exmaple 11-11: A fake Tombola doesn't go undetected

In [18]:
from tombola import Tombola

class Fake(Tombola):
    def pick(self):
        return 13

In [21]:
repr(Fake)

"<class '__main__.Fake'>"

In [22]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

## Subclassing the Tombola ABC

---
### Example 11-12: bingo.py: BingoCage is a  concrete subclass of Tombola (See file `bingo.py`)

In [26]:
from bingo import BingoCage

bingo = BingoCage((1, 2, 3, 4, 5, 6, 7, 8, 9, 10))
bingo.inspect()

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [27]:
bingo.pick()

9

In [28]:
bingo()

7

In [29]:
bingo.inspect()

(1, 2, 3, 4, 5, 6, 8, 10)

---
### Example 11-13: lotto.py: LotteryBlower is a concrete subclass that overrides the inspect and loaded methods from Tombola (See file `lotto.py`)

In [25]:
from lotto import LotteryBlower

In [32]:
lotto = LotteryBlower((1, 2, 3, 2.5, 1.2, 2.4))
lotto.inspect()

(1, 1.2, 2, 2.4, 2.5, 3)

In [33]:
lotto.pick()

2

In [34]:
lotto.pick()

2.4

In [35]:
lotto.inspect()

(1, 1.2, 2.5, 3)

## A virtual Subclass of Tombola

---
### Example 11-14: tombolist.py: class TomboList is a virtual subclass of Tombola (See file `tombolist.py`)

In [2]:
from tombola import Tombola
from tombolist import TomboList

issubclass(TomboList, Tombola)

True

In [3]:
t = TomboList(range(100))

In [4]:
isinstance(t, Tombola)

True

In [5]:
isinstance(t, TomboList)

True

In [8]:
repr(TomboList.__mro__)

"(<class 'tombolist.TomboList'>, <class 'list'>, <class 'object'>)"

## How the Tombola Subclasses Were Tested

---
### Example 11-15: tombola_runner.py: test runner for Tombola subclasses 